In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import Row
from pyspark.sql.types import IntegerType, StringType, StructType, StructField
from pyspark.sql import functions as F
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql.functions import col, dayofmonth, weekofyear, month, quarter, year, date_format


In [0]:
spark = SparkSession.builder.appName("OrderStatusLookup").getOrCreate()

In [0]:

schema = StructType([
    StructField("order_status_id", IntegerType(), True),
    StructField("order_status_description", StringType(), True)
])

In [0]:

order_status_lookup = {
    1: "Pending",
    2: "Processing",
    3: "Rejected",
    4: "Completed"
}

print(order_status_lookup)

{1: 'Pending', 2: 'Processing', 3: 'Rejected', 4: 'Completed'}


In [0]:

order_status_df = spark.createDataFrame(order_status_lookup.items(), ["order_status_id", "order_status_description"])

order_status_df.show()


+---------------+------------------------+
|order_status_id|order_status_description|
+---------------+------------------------+
|              1|                 Pending|
|              2|              Processing|
|              3|                Rejected|
|              4|               Completed|
+---------------+------------------------+



In [0]:
date_df = spark.sql("SELECT explode(sequence(to_date('2016-01-01'), to_date('2019-1-1'), interval 1 day)) AS date")

In [0]:
display(date_df)

In [0]:
date_df = date_df.withColumn("date_id", row_number().over(Window.orderBy("date")).astype("int"))

In [0]:
# Rearranging the columns so that date_id is first
date_df = date_df[['date_id', 'date']]
display(date_df)

In [0]:
# Extract day, week, month, quarter, year, and day name
date_df = date_df.withColumn('day', dayofmonth(col('date')))
date_df = date_df.withColumn('week', weekofyear(col('date')))
date_df = date_df.withColumn('month', month(col('date')))
date_df = date_df.withColumn('quarter', quarter(col('date')))
date_df = date_df.withColumn('year', year(col('date')))
date_df = date_df.withColumn('day_name', date_format(col('date'), 'EEEE'))
date_df = date_df.withColumn('month_name', date_format(col('date'), 'MMMM'))

if the year starts on a Thursday or is a leap year that starts on a Wednesday, that particular year will have 53 numbered weeks.

In [0]:
display(date_df)

In [0]:
order_status_df.write.mode("overwrite").saveAsTable("order_status_lookup")

In [0]:

# Set the Azure storage account key
spark.conf.set(
    "fs.azure.account.key.adlsbikestoreinterns.blob.core.windows.net",
    dbutils.secrets.get(scope="bikes-scope", key="account-key")
)

In [0]:
# Step 5: Save the DataFrame as a Delta table
order_status_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(
        f"wasbs://dlbikestorestage2@adlsbikestoreinterns.blob.core.windows.net/mostafa/status_lookup"
    )

In [0]:
date_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(
        f"wasbs://dlbikestorestage2@adlsbikestoreinterns.blob.core.windows.net/mostafa/date_lookup"
    )